In [ ]:
!pip install pytorch_lightning
!pip install tensorboardX

In [ ]:
from collections import namedtuple

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as td

import pytorch_lightning as pl

import tqdm
import json
import sklearn.metrics as sm

import tensorboardX as tb
import tensorflow as tf
import datetime, os

import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(31337)

## Create pairs (first track, subsequent track, time)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
DATA_DIR = "/content/drive/MyDrive/recsys-itmo-2023/seminar_05/"
# данные, собранные с помощью lightFM рекомендера (10,000 сессий)

In [ ]:
data = pd.read_json(DATA_DIR + "data.json", lines=True)

In [ ]:
Pair = namedtuple("Session", ["user", "start", "track", "time"])

# разбиваем сессии на пары
def get_pairs(user_data):
    pairs = []
    first = None
    for _, row in user_data.sort_values("timestamp").iterrows():
        if first is None:
            first = row["track"]
        else:
            pairs.append(Pair(row["user"], first, row["track"], row["time"]))
        
        if row["message"] == "last":
            first = None
    return pairs  # получаем для каждого пользователя пары прослушанных треков

In [ ]:
# будем учиться предсказывать время прослушивания второго трека в паре
# пользователь сам выбирает первый трек!

In [ ]:
pairs = pd.DataFrame(
    data
    .groupby("user")
    .apply(get_pairs)
    .explode()
    .values
    .tolist(),
    columns=["user", "start", "track", "time"]
)

In [ ]:
figure, ax = plt.subplots()
sns.histplot(pairs["time"], ax=ax)
pass

## Train Model

In [ ]:
rdm = np.random.random(len(pairs))
train_data = pairs[rdm < 0.8]
val_data = pairs[(rdm >= 0.8) & (rdm < 0.9)]
test_data = pairs[rdm >= 0.9]  # train - 0.9, test - 0.1

len(train_data), len(val_data), len(test_data)

In [ ]:
class ContextualRanker(pl.LightningModule):
    def __init__(self, embedding_dim=10):
        super().__init__()
        self.embedding_dim = embedding_dim  # размерность эмбеддинга
        
        # We won't have embeddings for everything, but that's ok
        # хранилища с эмбеддингами (всего N=50000 треков)
        self.context = nn.Embedding(num_embeddings=50000, embedding_dim=self.embedding_dim)
        self.track = nn.Embedding(num_embeddings=50000, embedding_dim=self.embedding_dim)
        # кол-во параметров: 1M
        # два разных эмбеддинга очим т.к. треки могут быть как стартовыми, так и первыми в сессии
        # 1. трек является стартовым
        # 2. трек не является стартовым

    def forward(self, x):
        # x - это пара
        context = self.context(x[:, 0])  # start track
        track = self.track(x[:, 1])  # next track
        # просто перемножаем два эмбеддинга?!
        return torch.sum(context * track, dim=1)  # скалярное произведение
            
    def step(self, batch, batch_idx, metric, prog_bar=False):
        # для каждого батча получаем предсказания
        x, y = batch
        predictions = self.forward(x)
        # MSE подходит для регрессии
        loss = F.mse_loss(predictions, y.float(), reduction='mean')
        self.log(metric, loss, prog_bar=prog_bar)  # логирование лосса
        return loss

    def test_step(self, batch, batch_idx, prog_bar=False):
        # во время тестирования считаем три разных лосса
        x, y = batch
        predictions = self.forward(x)
        targets = y[:, 0].float()  # таргеты
        avgs = y[:, 1].float()  # среднее
        rdms = y[:, 2].float()  # случайное

        loss = F.mse_loss(predictions, targets, reduction='mean')
        avg_loss = F.mse_loss(avgs, targets, reduction='mean')
        rdm_loss = F.mse_loss(rdms, targets, reduction='mean')

        self.log("test_loss", loss, prog_bar=prog_bar)  # логируем
        self.log("avg_loss", avg_loss, prog_bar=prog_bar)
        self.log("rdm_loss", rdm_loss, prog_bar=prog_bar)

    def training_step(self, batch, batch_idx):
        # шаг обучения
        return self.step(batch, batch_idx, "train_loss")
    
    def validation_step(self, batch, batch_idx):
        # шаг валидации
        return self.step(batch, batch_idx, "val_loss", True)
        
    def configure_optimizers(self):
        # используем адам, т.к. хорошо ведет себя с тяжелыми хвостами?
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3, weight_decay=1e-5)
        # уменьшаем LR если в течение трех последних эпох не происходило изменения лосса
        lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)
        scheduler = {
            'scheduler': lr_scheduler,
            'reduce_on_plateau': True,
            'monitor': 'val_loss'
        }
        return [optimizer], [scheduler]

In [ ]:
class ContextualRankerData(pl.LightningDataModule):
    def __init__(self, train_data, val_data, test_data, features):
        super().__init__()
        self.train_data = train_data
        self.val_data = val_data
        self.test_data = test_data
        self.features = features

    def prepare_data(self):
        """
        метод для скачивания и предобработки
            докидываем две доп. колонки для дальнейшего подсчета лосса и сравнения
        """
        self.test_data = self.test_data.assign(rdm = np.random.random(len(self.test_data))).assign(avg = self.train_data["time"].mean())

    def setup(self, stage=None):
        if stage == "fit" or stage is None:
        self.train_dataset = td.TensorDataset(
            torch.from_numpy(self.train_data[self.features].values),  # фичи
            torch.from_numpy(self.train_data["time"].values)  # таргеты
            )
        
        self.val_dataset = td.TensorDataset(
            torch.from_numpy(self.val_data[self.features].values), 
            torch.from_numpy(self.val_data["time"].values)
            )

        if stage == "test" or stage is None:  
        self.test_dataset = td.TensorDataset(
            torch.from_numpy(self.test_data[self.features].values),
            torch.from_numpy(self.test_data[["time", "avg", "rdm"]].values)  # закидываем случайное и среднее времена
        )
    def train_dataloader(self):
        # данные при делении на батчи лучше перемешивать
        return td.DataLoader(self.train_dataset, batch_size=2048, shuffle=True, num_workers=0)

    def val_dataloader(self):
        return td.DataLoader(self.val_dataset, batch_size=2048, num_workers=0)

    def test_dataloader(self):
        return td.DataLoader(self.test_dataset, batch_size=512, shuffle=False, num_workers=0)

In [ ]:
net = ContextualRanker(embedding_dim=100)
data_module = ContextualRankerData(train_data, val_data, test_data, features = ["start", "track"])

# checkpoint: сохраняем модель и выбираем лучшую по минимальному лоссу
checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor="val_loss")

trainer = pl.Trainer(
    max_epochs=300,
    accelerator='gpu', 
    devices=1,
    callbacks=[
        # останавливаем обучение, если 5 эпох ничего не меняется
        pl.callbacks.early_stopping.EarlyStopping(monitor="val_loss", patience=5),
        # логируем LR
        pl.callbacks.LearningRateMonitor(logging_interval="step"),
        checkpoint_callback
    ])

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/lightning_logs --host localhost

In [ ]:
trainer.fit(
    net,  # инстанс сети
    data_module  # инстанс данных
)  # обучаемся

In [ ]:
# сохраняем модельку
best = ContextualRanker.load_from_checkpoint(checkpoint_callback.best_model_path, embedding_dim=100)

In [ ]:
# тестируем: наша модель справляется лучше, чем рандомное и среднее предсказание
trainer.test(best, data_module)

## Compute top recommendations

In [ ]:
track_meta = pd.read_json(DATA_DIR + "tracks.json", lines=True)

In [ ]:
# для каждого трека по id хотим подобрать какое-то кол-во id ближайших соседей
context_embeddings = dict(best.named_parameters())["context.weight"].data.cpu().numpy()
track_embeddings = dict(best.named_parameters())["track.weight"].data.cpu().numpy()

In [ ]:
track_meta.head()

In [ ]:
k = 100
with open(DATA_DIR + "tracks_with_recs.json", "w") as rf:
    for _, track in tqdm.tqdm(track_meta.iterrows()):
        embedding = context_embeddings[track["track"]]  # эмбеддинг, когда является стартовым
        # ищем соседей по эмбеддингу из другого хранилища эмбеддингов track_embeddings
        neighbours = np.argpartition(-np.dot(track_embeddings, embedding), k)[:k]
        
        recommendation = dict(track)
        recommendation["recommendations"] = neighbours.tolist()
        
        rf.write(json.dumps(recommendation) + "\n")

In [ ]:
track = 3916
embedding = context_embeddings[track]
track_meta.loc[track_meta["track"] == track, ["artist", "title"]]

In [ ]:
k = 10
neighbours = np.argpartition(-np.dot(track_embeddings, embedding), k)[:k]
track_meta.loc[track_meta["track"].isin(neighbours), ["artist", "title"]]

In [ ]:
# сохраняем подсчитанные рекомендации